In [5]:
## Import necessary modules
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, matthews_corrcoef
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import seaborn as sns
import pandas as pd
import csv
import math
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score,KFold,StratifiedKFold,cross_val_predict
from sklearn.metrics import classification_report
import sklearn
import tensorflow as tf
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import Model
from keras.layers import LSTM, Dense, Bidirectional, LayerNormalization, BatchNormalization, Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PIPs/BTGA_Selected_Features.csv')

In [11]:
df.shape

In [12]:
from sklearn.preprocessing import StandardScaler
# Initialise the Scaler
scaler = StandardScaler()
# To scale data
scaler.fit(df)

StandardScaler()

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# 处理缺失值
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(999, inplace=True)

# 分割特征和标签
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

print(f"原始数据形状: {X.shape}")
print(f"原始类别分布: {Counter(y)}")

# 应用 SMOTE 进行过采样
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

print(f"SMOTE后数据形状: {X_res.shape}")
print(f"SMOTE后类别分布: {Counter(y_res)}")

# 标准化特征（在 SMOTE 之后）
scaler = StandardScaler()
X_res_scaled = scaler.fit_transform(X_res)

# 分割为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X_res_scaled, y_res,
    test_size=0.2,
    random_state=42,
    stratify=y_res
)

print(f"训练样本: {len(X_train)}, 测试样本: {len(X_test)}")
print(f"训练集类别分布: {Counter(y_train)}")
print(f"测试集类别分布: {Counter(y_test)}")

Self-Attention Bidirectional Temporal Convolutional Network

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, roc_auc_score
from keras.models import Sequential
from keras.layers import Bidirectional, Dense, MaxPooling1D
from keras.models import Model
from keras.optimizers import Adam
import tensorflow as tf
from keras import layers, regularizers
from keras.layers import Conv1D, Flatten, LayerNormalization, Dropout
from tensorflow.keras import layers, models

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam


class SelfAttention(layers.Layer):
    def __init__(self, attention_dim=16, dropout_rate=0.2, use_ffn=True, ffn_dim=32):
        super().__init__()
        self.attention_dim = attention_dim
        self.dropout_rate = dropout_rate
        self.use_ffn = use_ffn
        self.ffn_dim = ffn_dim

        self.drop_attn = Dropout(dropout_rate)

        # ✅ BatchNorm instead of LayerNorm
        self.bn1 = BatchNormalization()

        if use_ffn:
            self.ffn1 = Dense(ffn_dim, activation="relu")
            self.ffn2 = None  
            self.drop_ffn = Dropout(dropout_rate)
            self.bn2 = BatchNormalization()

    def build(self, input_shape):
        d_model = int(input_shape[-1])

        self.W_q = self.add_weight(
            name="W_q", shape=(d_model, self.attention_dim),
            initializer="glorot_uniform", trainable=True
        )
        self.W_k = self.add_weight(
            name="W_k", shape=(d_model, self.attention_dim),
            initializer="glorot_uniform", trainable=True
        )
        self.W_v = self.add_weight(
            name="W_v", shape=(d_model, self.attention_dim),
            initializer="glorot_uniform", trainable=True
        )
        self.W_o = self.add_weight(
            name="W_o", shape=(self.attention_dim, d_model),
            initializer="glorot_uniform", trainable=True
        )

        if self.use_ffn:
            self.ffn2 = Dense(d_model)

        super().build(input_shape)

    def call(self, x, training=None):
        Q = tf.matmul(x, self.W_q)
        K = tf.matmul(x, self.W_k)
        V = tf.matmul(x, self.W_v)

        scores = tf.matmul(Q, K, transpose_b=True)
        scores = scores / tf.math.sqrt(tf.cast(self.attention_dim, tf.float32))

        weights = tf.nn.softmax(scores, axis=-1)
        weights = self.drop_attn(weights, training=training)

        attn = tf.matmul(weights, V)
        attn = tf.matmul(attn, self.W_o)

        # ✅ Residual + BatchNorm
        y = self.bn1(x + attn, training=training)

        if self.use_ffn:
            f = self.ffn1(y)
            f = self.ffn2(f)
            f = self.drop_ffn(f, training=training)
            # ✅ Residual + BatchNorm
            y = self.bn2(y + f, training=training)

        return y


# -------------------
# TCN Block (BatchNorm instead of LayerNorm)
# -------------------
def TCNBlock(x, filters, kernel_size, dilation_rate, dropout_rate=0.2):
    x = Conv1D(filters, kernel_size, padding="causal",
               dilation_rate=dilation_rate, activation="linear")(x)
    x = BatchNormalization()(x)     # ✅ BatchNorm
    x = layers.Activation("relu")(x)
    x = Dropout(dropout_rate)(x)
    return x


# -------------------
# Hybrid Bi-TCN + Self-Attention Model
# -------------------
def create_bi_tcn_attention_model(
    input_shape,
    attention_dim=16,
    attn_dropout=0.2,
    use_ffn=True,
    ffn_dim=32
):
    inputs = layers.Input(shape=input_shape)

    # Forward TCN
    xf = TCNBlock(inputs, 64, 3, 1)
    xf = TCNBlock(xf,     64, 3, 2)
    xf = TCNBlock(xf,     32, 3, 4)

    # Backward TCN
    x_rev = layers.Lambda(lambda t: tf.reverse(t, axis=[1]))(inputs)
    xb = TCNBlock(x_rev, 64, 3, 1)
    xb = TCNBlock(xb,    64, 3, 2)
    xb = TCNBlock(xb,    32, 3, 4)
    xb = layers.Lambda(lambda t: tf.reverse(t, axis=[1]))(xb)

    # Merge
    x = layers.Add()([xf, xb])

    # Attention
    x = SelfAttention(
        attention_dim=attention_dim,
        dropout_rate=attn_dropout,
        use_ffn=use_ffn,
        ffn_dim=ffn_dim
    )(x)

    # Output
    x = Flatten()(x)
    outputs = Dense(1, activation="sigmoid")(x)

    return models.Model(inputs, outputs)


# -------------------
# Compile
# -------------------
input_shape = (X_train.shape[1], 1)
model = create_bi_tcn_attention_model(input_shape, attention_dim=16, attn_dropout=0.2, use_ffn=True, ffn_dim=32)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Initialize lists to store evaluation metrics for each fold
accuracy_list = []
f1_list = []
precision_list = []
recall_list = []
sensitivity_list = []
specificity_list = []
mcc_list = []
auc_list = []  # Initialize a list for AUC values

# Perform K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 在K-fold循环中
for train_index, val_index in kfold.split(X_train,y_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # 定义回调
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

    callbacks = [early_stop, reduce_lr]

    # 训练
    history = model.fit(X_train_fold, y_train_fold,
                       epochs=60,
                       batch_size=64,
                       verbose=0,
                       validation_split=0.1,
                       callbacks=callbacks)

    # 保持您原有的评估代码不变
    y_val_pred = model.predict(X_val_fold)
    y_val_pred_binary = (y_val_pred > 0.5).astype(int)

    # Calculate confusion matrix for the current fold
    cm = confusion_matrix(y_val_fold, y_val_pred_binary)
    # Calculate AUC for the current fold
    auc = roc_auc_score(y_val_fold, y_val_pred)  # Use predicted probabilities for AUC calculation
    auc_list.append(auc)


    # Calculate performance evaluation metrics for the current fold
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]

    # Accuracy
    accuracy = (TP + TN) / float(TP + TN + FP + FN)
    accuracy_list.append(accuracy)

    # F1-Score
    f1 = 2 * TP / float(2 * TP + FP + FN)
    f1_list.append(f1)

    # Precision
    precision = TP / float(TP + FP)
    precision_list.append(precision)

    # Recall
    recall = TP / float(TP + FN)
    recall_list.append(recall)

    # Sensitivity
    sensitivity = TP / float(TP + FN)
    sensitivity_list.append(sensitivity)

    # Specificity
    specificity = TN / float(TN + FP)
    specificity_list.append(specificity)

    # MCC (Matthews Correlation Coefficient)
    mcc = ((TP * TN) - (FP * FN)) / float((np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))) or 1)
    mcc_list.append(mcc)

# Calculate average performance evaluation metrics across all folds
avg_accuracy = np.mean(accuracy_list)
avg_f1 = np.mean(f1_list)
avg_precision = np.mean(precision_list)
avg_recall = np.mean(recall_list)
avg_sensitivity = np.mean(sensitivity_list)
avg_specificity = np.mean(specificity_list)
avg_mcc = np.mean(mcc_list)

# Calculate average AUC across all folds
avg_auc = np.mean(auc_list)
# Print average performance evaluation metrics
print("Accuracy =", avg_accuracy)
print("F1 Score =", avg_f1)
print("Precision =", avg_precision)
print("Recall =", avg_recall)
print("Sensitivity =", avg_sensitivity)
print("Specificity =", avg_specificity)
print("MCC =", avg_mcc)
print("AUC =", avg_auc)